# Lessons Learned and More Questions


## 1. Number of Parameters
Neural models are parametric models -  models possess fixed number of parameters (aka weights) - irrespective of number of training examples.
These weigthts are heart and soul of models. Neural models are awesome since all these weihgts can be tuned/adjusted in an end-to-end optimization task over a defined loss function. However we, programmers/scientists get to choose how many weights (aka parameters) a model should possess. That is the art of deep learning experts!

I can intuitively feel that there is a connection between number of training examples and the number of parameters, but havent figured out mathematical formulation of my intuition. 

In the content of sequence-to-sequence learning, I phrase these questions:
1. If we have only 1000 training examples, then how many weights in seq-seq models can we learn from these 1000 sequence pairs?
2. If I have 1000 weights in my seq-seq model, how many sequence pairs do I need to correctly and reliably set them during back propagation?


# 2. Bottlenecks

I view encoder-deocoder network as an information flow network. The input sequence (in descrete words) contains some information, then we take it to embedding space using embedder module, then we pass it through the encoder module, then we pass it through the decoder (i.e inverse encoder) module, then we pass it through the generator (i.e. inverse embedder) module which then produces words on the destination side. 

Question I am asking now, is how do we know the modules doesnt have bottlenecks and inverse bottlenecks? By Bottleneck I mean pipe for the flow is too small, i.e. vector dimension is too small, so we spill out some information which may actually be needed. By inverse-bottleneck, I mean we have too large vector dimensions, so the model fills in some random junk that is not needed, thus producing words which are not needed.


I did an experiment related to these questions.
My seq2seq model has BiLSTM encoder and LSTM decoder with configurable number of layers. No attention mechanism was used.
My test dataset was a simple copy task, it had 200 vocabulary types, with each sequence of length 12. The words were randomly generated.  
embedding size, hidden sizes were all 100 dimensions. 
With this setting, the model couldnt do copy task (1) no matter how much training data I randomly generate and train it on (what if we had infinite examples?). (2) even if I increase layers from 1 to 6, it could not learn to copy.

This is an harsh test because the words are randomly generated from uniform distribution. But in reality, natural language words occur as per power law (so throwing out rare words is probably okay).
- Can the 200 randomly distributed words be expressed in 100 dimensional vector space?
- Can this information be correctly transferred from source to destination using 100 dimensional vector space?

---

1. I suspect the 100 dimesional vector space is a **bottleneck** for my setting - it drops some necessary information, and hence the decoder is unable to reconstruct it. We might get lucky in realworld where some words are rare, the model might drop rare words and still appear as a good-enough.  Since here the numbers are uniformally distributed, every word is important, so the effect of bottleneck can be easily seen.
2. If I use 250 dimensions of embedding and hidden space, the model can learn to copy (within 100 epochs x 50 batches each x 50 sequences each). 
3. I couldnt find experimental evidance for inverse-bottleneck, that is adding bigger dimensions caused no trouble. (However my intuition is they need more examples)


This behaviour was alreadu studied by Nelson Liu el al and documented here  [LSTMs Exploit Linguistic Attributes of Data](https://homes.cs.washington.edu/~nfliu/papers/liu+levy+schwartz+tan+smith.repl4nlp2018.pdf)
--- 


## 3. Importance of Dropout

I trained a bunch of enc-dec models. I found that if I train longer (i.e. more epochs than needed), it overfits to the training set and the loss on validation set keeps going higher.

![](https://i.stack.imgur.com/eVFct.png)

(the image is not very accurate. Just imagine that after some point along the X axis (number of epochs) the loss on validation set keeps climbing high)
This is exactly where we need regularization.

And, surpisingly the solution is simple: just use drop outs! Use dropouts!

